<h1> Question 1 - Importation of tweeets</h1>

In [1]:
import GetOldTweets3 as got
import pandas as pd
import csv
from datetime import date, timedelta

In [5]:
with open("data/trump.csv", 'w', encoding="utf-8") as f:
    # Wrap the file object by the writer.
    writer = csv.writer(f)
    
    # Write the header line to the output if needed.
    writer.writerow(['date', 'tweet'])
    
    start = date(2019, 5, 1)
    end = date(2019, 5, 5)
    #end = date.today()
    day = timedelta(days=1)

    mydate = start
    
    while mydate < end:
        print("{date.day:02}{date.month:02}{date.year}".format(date=mydate))
        
        tweetCriteria = got.manager.TweetCriteria().setUsername("realDonaldTrump")\
                                           .setSince(("{date.year}-{date.month:02}-{date.day:02}".format(date=mydate)))\
                                           .setUntil(("{date.year}-{date.month:02}-{date.day:02}".format(date=mydate+day)))\
                                           .setTopTweets(True)\
                                           .setMaxTweets(240)
        
        mydate = mydate + day
        
    
        # Loop through the dictionary and get the tuples.
        # You may want to sort it somehow. Not shown here.
        for tweets in got.manager.TweetManager.getTweets(tweetCriteria):
            #writer.writerow(t)

            writer.writerow([tweets.date, tweets.text])

01052019
02052019
03052019
04052019


In [6]:
data = pd.read_csv("data/trump.csv")
data.head()

,date,tweet
0,2019-05-01 13:03:41+00:00,"NO COLLUSION, NO OBSTRUCTION. Besides, how can..."
1,2019-05-01 12:28:53+00:00,Why didn’t President Obama do something about ...
2,2019-05-01 12:06:10+00:00,"Congress must change the Immigration Laws now,..."
3,2019-05-01 11:11:42+00:00,“No President in history has endured such vici...
4,2019-05-01 11:01:15+00:00,Gallup Poll: 56% of Americans rate their finan...


<h1> Question 2 - Creation of a daily sentiment time serie</h1>

<h1> Question 3 - Properties of the sentiment index time series</h1>

<h1> Question 4 - Comparison bewtween tweets and S&P 500</h1>

<h1> Question 5 - Test for Granger causality</h1>